In [2]:
!pip install hf_xet
!pip install trl
!pip install --upgrade transformers
!pip install tensorboard
!pip install -U bitsandbytes
!pip install transformers_stream_generator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12426 sha256=c39cbed6937a57b30c0ee812bb43546ca737f93b1ec13e44d10922fdb6869ffc
  Stored in directory: /root/.cache/pip/wheels/23/e8/f0/b3c58c12d1ffe60bcc8c7d121115f26b2c1878653edfca48db
Successfully built transformers_stream_generator


KeyboardInterrupt: 

In [1]:
import os
from datasets import load_dataset, concatenate_datasets, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_LLM/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_LLM


In [4]:
# load tuning sets for <Sunrise on the Reaping>
sunrise_train_path = './reasoned_qa_output/sunrise_train_data.json'
sunrise_val_path = './reasoned_qa_output/sunrise_val_data.json'
sunrise_test_path = './reasoned_qa_output/sunrise_test_data.json'

sunrise_train_datasets = Dataset.from_json(sunrise_train_path)
sunrise_val_datasets = Dataset.from_json(sunrise_val_path)
sunrise_test_datasets = Dataset.from_json(sunrise_test_path)

In [5]:
# load tuning sets for <All Fours>
allfours_train_path = './reasoned_qa_output/allfours_train_data.json'
allfours_val_path = './reasoned_qa_output/allfours_val_data.json'
allfours_test_path = './reasoned_qa_output/allfours_test_data.json'

allfours_train_datasets = Dataset.from_json(allfours_train_path)
allfours_val_datasets = Dataset.from_json(allfours_val_path)
allfours_test_datasets = Dataset.from_json(allfours_test_path)

In [6]:
# Combine two books together
train_set = concatenate_datasets([sunrise_train_datasets, allfours_train_datasets])
val_set = concatenate_datasets([sunrise_val_datasets, allfours_val_datasets])
test_set = concatenate_datasets([sunrise_test_datasets, allfours_test_datasets])

In [7]:
# train_sample_size = train_set.num_rows // 5
# val_sample_size = val_set.num_rows // 5
# test_sample_size = test_set.num_rows // 5

In [8]:
# train_set = train_set.shuffle(seed=42).select(range(train_sample_size))
# val_set = val_set.shuffle(seed=42).select(range(val_sample_size))
# test_set = test_set.shuffle(seed=42).select(range(test_sample_size))

In [9]:
train_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 958
})

In [10]:
val_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 120
})

In [11]:
test_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 120
})

In [12]:
print(train_set["Question"][-1])

What trick does Jordi teach the protagonist to help stop thinking about Davey?


In [13]:
# Load Reasoning LLM
gemma_model = 'Qwen/Qwen3-4B'
# Load the specified tokenizer for the model to ensure consistency
tokenizer = AutoTokenizer.from_pretrained(gemma_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [14]:
# load model
model = AutoModelForCausalLM.from_pretrained(
    gemma_model,
    device_map="auto"  # instructs the transformers library to automatically distribute the model's layer cross available decives to optimise memory usage
)
model.config.use_cache = False

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/32.8k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [16]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    target_modules=["q_proj", "v_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 7,962,624 || all params: 4,030,430,720 || trainable%: 0.1976


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [17]:
def formatting_prompts_func(examples):
    texts = []
    for question, answer, reasoning_list in zip(examples["Question"], examples["Answer"], examples["Reasoning"]):
        reasoning_str = "\n".join([f"Step {i+1}: {step}" for i, step in enumerate(reasoning_list)])
        formatted_text = (
            f"### Question:\n{question}\n\n"
            f"### Thought Process:\n"
            f"{reasoning_str}\n\n"
            f"### Answer:\n{answer}"
        )
        texts.append(formatted_text)
    return {"text": texts}

In [18]:
train_dataset_formatted = train_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=train_set.column_names
)
validation_dataset_formatted = val_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=val_set.column_names
)

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [19]:
print(train_dataset_formatted['text'][-1])

### Question:
What trick does Jordi teach the protagonist to help stop thinking about Davey?

### Thought Process:
Step 1: During their meeting, Jordi demonstrates a technique she used to quit smoking.
Step 2: The protagonist snaps a rubber band on her wrist each time she thinks of Davey.
Step 3: This method aims to reduce obsessive thoughts through punishment.

### Answer:
Jordi teaches her to use a rubber band for negative reinforcement whenever she thinks about him.


In [20]:
import inspect
from trl import SFTTrainer
print(inspect.signature(SFTTrainer.__init__))

(self, model: Union[str, torch.nn.modules.module.Module, transformers.modeling_utils.PreTrainedModel], args: Union[trl.trainer.sft_config.SFTConfig, transformers.training_args.TrainingArguments, NoneType] = None, data_collator: Optional[transformers.data.data_collator.DataCollator] = None, train_dataset: Union[datasets.arrow_dataset.Dataset, datasets.iterable_dataset.IterableDataset, NoneType] = None, eval_dataset: Union[datasets.arrow_dataset.Dataset, dict[str, datasets.arrow_dataset.Dataset], NoneType] = None, processing_class: Union[transformers.tokenization_utils_base.PreTrainedTokenizerBase, transformers.image_processing_utils.BaseImageProcessor, transformers.feature_extraction_utils.FeatureExtractionMixin, transformers.processing_utils.ProcessorMixin, NoneType] = None, compute_loss_func: Optional[Callable] = None, compute_metrics: Optional[Callable[[transformers.trainer_utils.EvalPrediction], dict]] = None, callbacks: Optional[list[transformers.trainer_callback.TrainerCallback]] 

In [21]:
# SFTConfig 应该包含原 TrainingArguments 的参数，以及 SFT 特有的参数。
sft_config = SFTConfig(
    output_dir="./results", # From TrainingArguments
    num_train_epochs=3, # From TrainingArguments
    per_device_train_batch_size=4, # From TrainingArguments
    per_device_eval_batch_size=4,
    warmup_steps=500,
    gradient_accumulation_steps=1, # From TrainingArguments
    learning_rate=2e-4, # From TrainingArguments
    logging_steps=10, # From TrainingArguments
    save_steps=500, # From TrainingArguments
    # Add SFT-specific parameters here:
    max_seq_length=128, # Now correctly placed in SFTConfig
    dataset_text_field="text", # Important for SFTTrainer
    packing=True, # Recommended for efficient SFT
    optim="adafactor",
    fp16=True, # For mixed precision, if SFTConfig handles this
)

# 注意：现在 SFTTrainer 只需要一个 `args` 参数，就是您的 sft_config
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer, # Keep this, as it's separate from args
    train_dataset=train_dataset_formatted,
    eval_dataset=validation_dataset_formatted,
    peft_config=lora_config,
    args=sft_config, # <-- Pass the SFTConfig object here!
    # formatting_func=formatting_func, # If you have a custom formatting function
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:404: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:450: UserWarning: You are using packing, but the attention implementation is not set to 'flash_attention_2'. Packing flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that re

Adding EOS to train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
import torch
torch.cuda.empty_cache()

In [23]:
print("Model Tuning Start...")
trainer.train()
print("Model Tuning Completed")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Model Tuning Start...


wandb: Currently logged in as: xingze-su-2024 (xingze-su-2024-singapore-management-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.105500
20,3.031400
30,3.007400
40,2.858300
50,2.682500
60,2.561400
70,2.509400
80,2.260000
90,2.311800
100,2.260000


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


Model Tuning Completed


In [24]:
# Save LoAR Adapter
output_model_path = "./LoRA_Qwen"
trainer.save_model(output_model_path)
print(f"Fine-tuned LoRA Adapter saved to: {output_model_path}")

Fine-tuned LoRA Adapter saved to: ./LoRA_Qwen
